In [2]:
import pandas as pd
import numpy as np
import time

# Generación del dataset para el entrenamiento del modelo

## Datasets de puntajes

Cargamos data descargada de Kaggle (https://www.kaggle.com/datasets/gabrielmanfredi/football-players-ratings).

Vamos a usar un solo torneo para entrenar el modelo, nos quedamos con la Premier League 2017-2018

In [3]:
ratings = pd.read_csv("data/data_football_ratings.csv")
ratings = ratings[ratings["competition"] == "Premier League 2017-18"]
ratings = ratings[['match', 'player', 'original_rating', 'rater', 'is_human', 'team']]
ratings['match'] = ratings['match'].str.split(',').str[0]
ratings.head()

,match,player,original_rating,rater,is_human,team
4148,Arsenal - Leicester,Alex Oxlade-Chamberlain,9.23,WhoScored,0,Arsenal
4149,Arsenal - Leicester,Alex Oxlade-Chamberlain,7.00,SkySports,1,Arsenal
4150,Arsenal - Leicester,Nacho Monreal,7.30,WhoScored,0,Arsenal
4151,Arsenal - Leicester,Nacho Monreal,5.00,SkySports,1,Arsenal
4152,Arsenal - Leicester,Hector Bellerin,7.01,WhoScored,0,Arsenal


Hay dos ratings por jugador, uno dado por el software de WhoScored y otro por un periodista de SkySports.  
Nuestro target será el promedio.

In [4]:
ratings = ratings.groupby(['player', 'team', 'match']).agg({'original_rating': 'mean'}).reset_index()

ratings = ratings.rename(columns={'original_rating': 'rating'})

ratings.head()

,player,team,match,rating
0,Aaron Cresswell,West Ham,Arsenal - West Ham,5.840
1,Aaron Cresswell,West Ham,Bournemouth - West Ham,6.215
2,Aaron Cresswell,West Ham,Brighton - West Ham,6.640
3,Aaron Cresswell,West Ham,Burnley - West Ham,6.120
4,Aaron Cresswell,West Ham,Chelsea - West Ham,5.980


## Leemos la tabla de resultados de FBRef y extraemos los links de los reportes

In [5]:
url = "https://fbref.com/en/comps/9/2017-2018/schedule/2017-2018-Premier-League-Scores-and-Fixtures"

fixture_links = pd.read_html(url, extract_links = "body")[0]

In [6]:
links = fixture_links.apply(lambda col: [v[0] if v[1] is None else f'https://fbref.com{v[1]}' for v in  col])["Match Report"]
links

0      https://fbref.com/en/matches/e3c3ddf0/Arsenal-...
1      https://fbref.com/en/matches/60f6cc1d/Watford-...
2      https://fbref.com/en/matches/2d369d17/Crystal-...
3      https://fbref.com/en/matches/684f704a/West-Bro...
4      https://fbref.com/en/matches/71b00bca/Chelsea-...
                             ...                        
418    https://fbref.com/en/matches/7c431214/Huddersf...
419    https://fbref.com/en/matches/8bb22e44/Newcastl...
420    https://fbref.com/en/matches/c137bab4/Manchest...
421    https://fbref.com/en/matches/db4a1124/Southamp...
422    https://fbref.com/en/matches/f8c06939/Tottenha...
Name: Match Report, Length: 423, dtype: object

In [7]:
fixture = pd.read_html(url)[0]
fixture["Match Report"] = links
fixture.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2017-08-11,19:45,Arsenal,2.5,4–3,1.5,Leicester City,59387.0,Emirates Stadium,Mike Dean,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,NaN
1,1.0,Sat,2017-08-12,12:30,Watford,2.1,3–3,2.6,Liverpool,20407.0,Vicarage Road Stadium,Anthony Taylor,https://fbref.com/en/matches/60f6cc1d/Watford-...,NaN
2,1.0,Sat,2017-08-12,15:00,Crystal Palace,1.1,0–3,1.5,Huddersfield,25448.0,Selhurst Park,Jonathan Moss,https://fbref.com/en/matches/2d369d17/Crystal-...,NaN
3,1.0,Sat,2017-08-12,15:00,West Brom,1.3,1–0,0.5,Bournemouth,25011.0,The Hawthorns,Robert Madley,https://fbref.com/en/matches/684f704a/West-Bro...,NaN
4,1.0,Sat,2017-08-12,15:00,Chelsea,1.5,2–3,0.6,Burnley,41616.0,Stamford Bridge,Craig Pawson,https://fbref.com/en/matches/71b00bca/Chelsea-...,NaN


Hay que corregir los nombres de los equipos para que coincidan con la tabla de ratings

In [8]:
reemplazos = {
    'West Brom': 'West Bromwich Albion',
    'Stoke City': 'Stoke',
    'Swansea City': 'Swansea',
    'Leicester City': 'Leicester',
    'Newcastle Utd': 'Newcastle United',
    'Manchester Utd': 'Manchester United'
}

fixture['Home'] = fixture['Home'].replace(reemplazos)
fixture['Away'] = fixture['Away'].replace(reemplazos)

In [10]:
fixture = fixture[['Home', 'Away', 'Match Report', 'Score']]
fixture['match'] = fixture['Home'] + ' - ' + fixture['Away']
fixture.drop(['Home', 'Away'], axis=1, inplace=True)
fixture.dropna(inplace=True)
fixture.head()

C:\Users\nazar\AppData\Local\Temp\ipykernel_8456\3665840221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixture['match'] = fixture['Home'] + ' - ' + fixture['Away']
C:\Users\nazar\AppData\Local\Temp\ipykernel_8456\3665840221.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixture.drop(['Home', 'Away'], axis=1, inplace=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_8456\3665840221.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

,Match Report,Score,match
0,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,4–3,Arsenal - Leicester
1,https://fbref.com/en/matches/60f6cc1d/Watford-...,3–3,Watford - Liverpool
2,https://fbref.com/en/matches/2d369d17/Crystal-...,0–3,Crystal Palace - Huddersfield
3,https://fbref.com/en/matches/684f704a/West-Bro...,1–0,West Bromwich Albion - Bournemouth
4,https://fbref.com/en/matches/71b00bca/Chelsea-...,2–3,Chelsea - Burnley


In [11]:
fixture.to_csv("data/links.csv", index=False)

## Extraemos las tablas de rendimiento

In [12]:
links = pd.read_csv("data/links.csv")
links.head()

,Match Report,Score,match
0,https://fbref.com/en/matches/e3c3ddf0/Arsenal-...,4–3,Arsenal - Leicester
1,https://fbref.com/en/matches/60f6cc1d/Watford-...,3–3,Watford - Liverpool
2,https://fbref.com/en/matches/2d369d17/Crystal-...,0–3,Crystal Palace - Huddersfield
3,https://fbref.com/en/matches/684f704a/West-Bro...,1–0,West Bromwich Albion - Bournemouth
4,https://fbref.com/en/matches/71b00bca/Chelsea-...,2–3,Chelsea - Burnley


In [23]:
def limpiar_tabla(tabla, nombre_tabla, match, score):
    # standard
    if nombre_tabla == 'home_standard':
        to_keep_as_is = ['Unnamed', 'Performance', 'Expected', 'SCA', 'Carries']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla = tabla.iloc[:, :-8]
        tabla.drop(columns=['Touches', 'Blocks', 'Int', 'Tkl', 'CrdY', 'CrdR', 'xAG'], inplace=True)
    elif nombre_tabla == 'away_standard':
        to_keep_as_is = ['Unnamed', 'Performance', 'Expected', 'SCA', 'Carries']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla = tabla.iloc[:, :-8]
        tabla.drop(columns=['Touches', 'Blocks', 'Int', 'Tkl', 'CrdY', 'CrdR', 'xAG'], inplace=True)
    # pass
    elif nombre_tabla == 'home_pass':
        to_keep_as_is = ['Unnamed']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Total-Cmp', 'Total-Att', 'Total-Cmp%', 'Short-Cmp%', 'Medium-Cmp%', 'Long-Cmp%'], inplace=True)
    elif nombre_tabla == 'away_pass':
        to_keep_as_is = ['Unnamed']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Total-Cmp', 'Total-Att', 'Total-Cmp%', 'Short-Cmp%', 'Medium-Cmp%', 'Long-Cmp%'], inplace=True)
    # pass types
    elif nombre_tabla == 'home_pass_types':
        to_keep_as_is = ['Unnamed', 'Pass Types', 'Corner Kicks', 'Outcomes']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['In', 'Out', 'Str', 'Cmp', 'Off', 'Blocks'], inplace=True)
    elif nombre_tabla == 'away_pass_types':
        to_keep_as_is = ['Unnamed', 'Pass Types', 'Corner Kicks', 'Outcomes']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['In', 'Out', 'Str', 'Cmp', 'Off', 'Blocks'], inplace=True)
    # defense
    elif nombre_tabla == 'home_defense':
        to_keep_as_is = ['Unnamed']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Tackles-Def 3rd', 'Tackles-Mid 3rd', 'Tackles-Att 3rd', 'Challenges-Tkl%', 'Tkl+Int'], inplace=True)
    elif nombre_tabla == 'away_defense':
        to_keep_as_is = ['Unnamed']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Tackles-Def 3rd', 'Tackles-Mid 3rd', 'Tackles-Att 3rd', 'Challenges-Tkl%', 'Tkl+Int'], inplace=True)
    # possession
    elif nombre_tabla == 'home_possession':
        to_keep_as_is = ['Unnamed']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Touches-Touches', 'Touches-Live', 'Take-Ons-Succ%', 'Take-Ons-Tkld', 'Take-Ons-Tkld%'], inplace=True)
    elif nombre_tabla == 'away_possession':
        to_keep_as_is = ['Unnamed']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Touches-Touches', 'Touches-Live', 'Take-Ons-Succ%', 'Take-Ons-Tkld', 'Take-Ons-Tkld%'], inplace=True)
    # misc
    elif nombre_tabla == 'home_misc':
        to_keep_as_is = ['Unnamed', 'Performance']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Crs', 'Int', 'TklW', 'Aerial Duels-Won%'], inplace=True)
    elif nombre_tabla == 'away_misc':
        to_keep_as_is = ['Unnamed', 'Performance']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Crs', 'Int', 'TklW', 'Aerial Duels-Won%'], inplace=True)
    # gk
    elif nombre_tabla == 'home_gk':
        to_keep_as_is = ['Unnamed', 'Shot Stopping']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Save%', 'Launched-Cmp%', 'Passes-Launch%', 'Goal Kicks-Launch%', 'Crosses-Stp%'], inplace=True)
    elif nombre_tabla == 'away_gk':
        to_keep_as_is = ['Unnamed', 'Shot Stopping']
        tabla.columns = ['{}-{}'.format(c[0], c[1]) if all(x not in c[0] for x in to_keep_as_is) else c[1] for c in tabla.columns]
        tabla.drop(columns=['Save%', 'Launched-Cmp%', 'Passes-Launch%', 'Goal Kicks-Launch%', 'Crosses-Stp%'], inplace=True)
    
    if 'home' in nombre_tabla:
        tabla['team'] = match.split(' - ')[0]
        tabla['team_goals'] = int(score.split('–')[0])
        tabla['conceded_goals'] = int(score.split('–')[1])
        tabla['win'] = int(score.split('–')[0]) > int(score.split('–')[1])
        tabla['tie'] = int(score.split('–')[0]) == int(score.split('–')[1])
    else:
        tabla['team'] = match.split(' - ')[1]
        tabla['team_goals'] = int(score.split('–')[1])
        tabla['conceded_goals'] = int(score.split('–')[0])
        tabla['win'] = int(score.split('–')[1]) > int(score.split('–')[0])
        tabla['tie'] = int(score.split('–')[1]) == int(score.split('–')[0])

    if nombre_tabla != 'home_gk' and nombre_tabla != 'away_gk':
        tabla.drop(tabla.index[-1], inplace=True)
    
    return tabla

Leamos las primeras 5 fechas, con eso deberia ser suficiente para entrenar un buen modelo.

In [24]:
tablas_clean = []

In [27]:
for i, row in links[:50].iterrows():
    match = row['match']
    link = row['Match Report']
    score = row['Score']
    tablas = pd.read_html(link)
    nombres_tablas = ['home_standard', 'home_pass', 'home_pass_types', 'home_defense', 'home_possession', 'home_misc', 'home_gk',
                      'away_standard', 'away_pass', 'away_pass_types', 'away_defense', 'away_possession', 'away_misc', 'away_gk']
    for tabla, nombre_tabla in zip(tablas[3:17], nombres_tablas):
        tabla = limpiar_tabla(tabla, nombre_tabla, match, score)
        tabla['match'] = match
        tablas_clean.append(tabla)
    time.sleep(10)

URLError: <urlopen error [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto>

Me olvidé lo de la posición!

In [31]:
def primera_posicion(pos_str):
    if pd.isna(pos_str):
        return np.nan  # Devolver NaN si el valor es NaN
    # Tomar la primera posición
    return pos_str.split(',')[0]

for df in tablas_clean:
    try:
        df['Pos'] = df['Pos'].apply(primera_posicion)
    except:
        pass

## Combinamos las tablas en una sola

In [32]:
from functools import reduce

def merge_and_concat(tablas_clean, group_size=7):
    merged_dfs = []
    
    for i in range(0, len(tablas_clean), group_size):
        group = tablas_clean[i:i + group_size]
        
        df_merged = reduce(
            lambda left, right: pd.merge(left, right, how='left', on=['Player', 'Nation', 'Age', 'team', 'match'], suffixes=('', '_dup')),
            group
        )
        
        df_merged = df_merged.loc[:, ~df_merged.columns.str.endswith('_dup')]
        
        merged_dfs.append(df_merged)
    
    df_final = pd.concat(merged_dfs, ignore_index=True)
    
    return df_final

df_final = merge_and_concat(tablas_clean, group_size=7)

df_final.head()


,Player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,Launched-Att,Passes-Att (GK),Passes-Thr,Passes-AvgLen,Goal Kicks-Att,Goal Kicks-AvgLen,Crosses-Opp,Crosses-Stp,Sweeper-#OPA,Sweeper-AvgDist
0,Alexandre Lacazette,9.0,fr FRA,FW,26-075,90,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Danny Welbeck,23.0,eng ENG,AM,26-258,74,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Theo Walcott,14.0,eng ENG,RW,28-148,16,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mesut Özil,11.0,de GER,AM,28-300,90,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Granit Xhaka,29.0,ch SUI,CM,24-318,90,0,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_final.to_csv("data/rendimientos_training.csv", index=False)

In [35]:
df_final.rename(columns={'Player': 'player'}, inplace=True)

In [36]:
df_final_final = df_final.merge(ratings, on=['match', 'team', 'player'], how='left')
df_final_final.head()

,player,#,Nation,Pos,Age,Min,Gls,Ast,PK,PKatt,...,Passes-Att (GK),Passes-Thr,Passes-AvgLen,Goal Kicks-Att,Goal Kicks-AvgLen,Crosses-Opp,Crosses-Stp,Sweeper-#OPA,Sweeper-AvgDist,rating
0,Alexandre Lacazette,9.0,fr FRA,FW,26-075,90,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.235
1,Danny Welbeck,23.0,eng ENG,AM,26-258,74,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.850
2,Theo Walcott,14.0,eng ENG,RW,28-148,16,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.280
3,Mesut Özil,11.0,de GER,AM,28-300,90,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Granit Xhaka,29.0,ch SUI,CM,24-318,90,0,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.740


## Guardamos el dataset

SI LEEMOS MÁS FECHAS NO SOBREESCRIBIR EL ARCHIVO

In [37]:
# df_final_final.to_csv("data/training_data.csv", index=False)

In [39]:
ratings.to_csv("data/ratings_training.csv", index=False)